<a href="https://colab.research.google.com/github/jovansgit/Hourly-Energy-Consumption/blob/main/Energy_Weather_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-14 23:34:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.93MB/s    in 0.2s    

2022-11-14 23:34:04 (4.93 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [15]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://my-bootcamp-jovan.s3.amazonaws.com/Chicago_Weather_Data.csv"
spark.sparkContext.addFile(url)
weather_data_df = spark.read.csv(SparkFiles.get("Chicago_Weather_Data.csv"), sep=",", header=True, inferSchema=True)

In [16]:
weather_data_df.show()

+----------+-------------------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|        dt|             dt_iso|timezone|city_name|      lat|       lon| temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|
+----------+-------------------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|1293840000|2011-01-01 00:00:00|  -21600|  Chicago|41.878114|-87.629798|48.96|     10000|    47.03|     44

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://my-bootcamp-jovan.s3.amazonaws.com/COMED_hourly.csv"
spark.sparkContext.addFile(url)
Energy_data_df = spark.read.csv(SparkFiles.get("COMED_hourly.csv"), sep=",", header=True, inferSchema=True)

In [8]:
Energy_data_df.show()

+-------------------+--------+
|           Datetime|COMED_MW|
+-------------------+--------+
|2011-12-31 01:00:00|  9970.0|
|2011-12-31 02:00:00|  9428.0|
|2011-12-31 03:00:00|  9059.0|
|2011-12-31 04:00:00|  8817.0|
|2011-12-31 05:00:00|  8743.0|
|2011-12-31 06:00:00|  8735.0|
|2011-12-31 07:00:00|  8993.0|
|2011-12-31 08:00:00|  9363.0|
|2011-12-31 09:00:00|  9545.0|
|2011-12-31 10:00:00|  9676.0|
|2011-12-31 11:00:00|  9937.0|
|2011-12-31 12:00:00| 10139.0|
|2011-12-31 13:00:00| 10326.0|
|2011-12-31 14:00:00| 10359.0|
|2011-12-31 15:00:00| 10293.0|
|2011-12-31 16:00:00| 10240.0|
|2011-12-31 17:00:00| 10416.0|
|2011-12-31 18:00:00| 11225.0|
|2011-12-31 19:00:00| 11907.0|
|2011-12-31 20:00:00| 11812.0|
+-------------------+--------+
only showing top 20 rows



In [17]:
weather_data_df = weather_data_df.withColumnRenamed("dt_iso","Datetime")

In [18]:
weather_data_df.show()

+----------+-------------------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|        dt|           Datetime|timezone|city_name|      lat|       lon| temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|
+----------+-------------------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|1293840000|2011-01-01 00:00:00|  -21600|  Chicago|41.878114|-87.629798|48.96|     10000|    47.03|     44

In [21]:
#Join the two DataFrame
joined_df = Energy_data_df.join(weather_data_df,on="Datetime", how="inner")
joined_df.show()

+-------------------+--------+----------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|           Datetime|COMED_MW|        dt|timezone|city_name|      lat|       lon| temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|
+-------------------+--------+----------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|2011-01-01 01:00:00|  9631.0|1293843600|  -21600|  Chicago|41.878114|-87.62979

In [22]:
#Drop null values
dropna_df = joined_df.dropna()
dropna_df.show()



+--------+--------+---+--------+---------+---+---+----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|Datetime|COMED_MW| dt|timezone|city_name|lat|lon|temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|
+--------+--------+---+--------+---------+---+---+----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
+--------+--------+---+--------+---------+---+---+----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-

In [23]:
cleaned_df = joined_df
cleaned_df.show()

+-------------------+--------+----------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|           Datetime|COMED_MW|        dt|timezone|city_name|      lat|       lon| temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|
+-------------------+--------+----------+--------+---------+---------+----------+-----+----------+---------+----------+--------+--------+--------+---------+----------+--------+----------+--------+---------+-------+-------+-------+-------+----------+----------+------------+-------------------+------------+
|2011-01-01 01:00:00|  9631.0|1293843600|  -21600|  Chicago|41.878114|-87.62979

In [52]:
#Create user dataframe for energy_consumption table
energy_consumption_df = cleaned_df.select(["Datetime", "COMED_MW"])
energy_consumption_df.show()

+-------------------+--------+
|           Datetime|COMED_MW|
+-------------------+--------+
|2011-12-31 01:00:00|  9970.0|
|2011-12-31 01:00:00|  9970.0|
|2011-12-31 02:00:00|  9428.0|
|2011-12-31 03:00:00|  9059.0|
|2011-12-31 04:00:00|  8817.0|
|2011-12-31 04:00:00|  8817.0|
|2011-12-31 05:00:00|  8743.0|
|2011-12-31 06:00:00|  8735.0|
|2011-12-31 07:00:00|  8993.0|
|2011-12-31 08:00:00|  9363.0|
|2011-12-31 09:00:00|  9545.0|
|2011-12-31 10:00:00|  9676.0|
|2011-12-31 11:00:00|  9937.0|
|2011-12-31 12:00:00| 10139.0|
|2011-12-31 13:00:00| 10326.0|
|2011-12-31 14:00:00| 10359.0|
|2011-12-31 15:00:00| 10293.0|
|2011-12-31 16:00:00| 10240.0|
|2011-12-31 17:00:00| 10416.0|
|2011-12-31 18:00:00| 11225.0|
+-------------------+--------+
only showing top 20 rows



In [57]:
#Create user dataframe for weather_df
weather_df = cleaned_df.select(["Datetime", "temp","visibility","dew_point", "feels_like","temp_min", "temp_max", "pressure", "humidity", "wind_speed", "wind_deg", "wind_gust", "rain_1h","rain_3h","snow_1h","clouds_all","weather_main","weather_description"])
weather_df.show()

+-------------------+-----+----------+---------+----------+--------+--------+--------+--------+----------+--------+---------+-------+-------+-------+----------+------------+-------------------+
|           Datetime| temp|visibility|dew_point|feels_like|temp_min|temp_max|pressure|humidity|wind_speed|wind_deg|wind_gust|rain_1h|rain_3h|snow_1h|clouds_all|weather_main|weather_description|
+-------------------+-----+----------+---------+----------+--------+--------+--------+--------+----------+--------+---------+-------+-------+-------+----------+------------+-------------------+
|2011-01-01 01:00:00|49.17|     10000|    47.25|     43.59|   44.64|   55.11|    1005|      93|     14.99|     170|    24.16|   null|   null|   null|       100|      Clouds|    overcast clouds|
|2011-01-01 02:00:00| 49.1|     10000|    47.17|     45.21|   44.53|   55.47|    1005|      93|      9.17|     180|     null|    0.2|   null|   null|       100|        Rain|         light rain|
|2011-01-01 03:00:00|47.84|   

In [56]:
energy_consumption_df.show()

+-------------------+--------+
|          date_time|COMED_MW|
+-------------------+--------+
|2011-12-31 01:00:00|  9970.0|
|2011-12-31 01:00:00|  9970.0|
|2011-12-31 02:00:00|  9428.0|
|2011-12-31 03:00:00|  9059.0|
|2011-12-31 04:00:00|  8817.0|
|2011-12-31 04:00:00|  8817.0|
|2011-12-31 05:00:00|  8743.0|
|2011-12-31 06:00:00|  8735.0|
|2011-12-31 07:00:00|  8993.0|
|2011-12-31 08:00:00|  9363.0|
|2011-12-31 09:00:00|  9545.0|
|2011-12-31 10:00:00|  9676.0|
|2011-12-31 11:00:00|  9937.0|
|2011-12-31 12:00:00| 10139.0|
|2011-12-31 13:00:00| 10326.0|
|2011-12-31 14:00:00| 10359.0|
|2011-12-31 15:00:00| 10293.0|
|2011-12-31 16:00:00| 10240.0|
|2011-12-31 17:00:00| 10416.0|
|2011-12-31 18:00:00| 11225.0|
+-------------------+--------+
only showing top 20 rows



# **LOAD**

In [58]:
#Change Datetime column name
weather_df = weather_df.withColumnRenamed("Datetime","date_time")

In [55]:
#Change Datetime column name
energy_consumption_df = energy_consumption_df.withColumnRenamed("Datetime","date_time")

In [30]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://energyconsumptiondb.cnnrufx43ak5.us-east-1.rds.amazonaws.com:5432/EnergyConsumption"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [60]:
# Write DataFrame to active_user table in RDS
weather_df.write.jdbc(url=jdbc_url, table='weather_table', mode=mode, properties=config)

In [63]:
# Write DataFrame to active_user table in RDS
energy_consumption_df.write.jdbc(url=jdbc_url, table='energy_table', mode=mode, properties=config)